In [1]:
import json
from dataclasses import dataclass
from typing import List, Tuple, Optional
from pyproj import Transformer

In [3]:
@dataclass
class Point:
    """
    Data class representing a survey point with its coordinates and relationships.

    Attributes:
        name (str): Identifier of the point (e.g., "KNUST TREK 10/2021/1")
        x_coord (float): X coordinate in Ghana National Grid system
        y_coord (float): Y coordinate in Ghana National Grid system
        bearing_to_next (Optional[str]): Bearing to the next point in DMS format (e.g., "13°10'")
        distance_to_next (Optional[float]): Distance to the next point in meters
        next_point (Optional[str]): Identifier of the next connected point
        latitude (Optional[float]): Converted latitude in decimal degrees
        longitude (Optional[float]): Converted longitude in decimal degrees
    """

    name: str
    x_coord: float
    y_coord: float
    bearing_to_next: Optional[str] = None
    distance_to_next: Optional[float] = None
    next_point: Optional[str] = None
    latitude: Optional[float] = None
    longitude: Optional[float] = None


class LandDataProcessor:
    """
    Processes land survey data and converts coordinates between Ghana National Grid and WGS84.

    This class handles the conversion of coordinates, processing of bearings and distances,
    and organization of survey data into a structured format.
    """

    def __init__(self):
        """
        Initialize the processor with coordinate transformation settings.
        Sets up the coordinate transformer from Ghana National Grid (EPSG:25000) to WGS84 (EPSG:4326).
        """
        # Initialize coordinate transformer for Ghana National Grid to WGS84
        self.transformer = Transformer.from_crs(
            "epsg:25000", "epsg:4326", always_xy=True
        )

    def convert_dms_to_decimal(self, dms_str: str) -> float:
        """
        Convert bearing from Degrees-Minutes-Seconds (DMS) format to decimal degrees.

        Args:
            dms_str (str): Bearing in DMS format (e.g., "13°10'")

        Returns:
            float: Bearing in decimal degrees

        Examples:
            >>> convert_dms_to_decimal("13°10'")
            13.166666666666666
        """
        if not dms_str:
            return 0.0

        # Split the DMS string into components
        parts = dms_str.replace("°", " ").replace("'", " ").strip().split()
        degrees = float(parts[0])
        minutes = float(parts[1]) if len(parts) > 1 else 0
        return degrees + (minutes / 60)

    def ghana_grid_to_latlon(
        self, easting: float, northing: float
    ) -> Tuple[float, float]:
        """
        Convert coordinates from Ghana National Grid to WGS84 latitude/longitude.

        Args:
            easting (float): Easting coordinate in Ghana National Grid
            northing (float): Northing coordinate in Ghana National Grid

        Returns:
            Tuple[float, float]: (latitude, longitude) in decimal degrees
        """
        # Transform coordinates using the initialized transformer
        lon, lat = self.transformer.transform(easting, northing)
        return (lat, lon)

    def process_land_data(self, data: dict) -> dict:
        """
        Process land survey data and convert coordinates.

        Args:
            data (dict): Input JSON data containing survey information

        Returns:
            dict: Processed data including converted coordinates and structured survey information

        The returned dictionary contains:
        - Plot information (number, area, location details)
        - Survey points with original and converted coordinates
        - Boundary points with converted coordinates
        - Bearings and distances between points
        """
        # Extract coordinate data from the input
        plan_data = data["site_plan_data"]["plan_data"]
        points: List[Point] = []

        # Process each survey point
        for i in range(len(plan_data["from"])):
            # Only process points with valid coordinates
            if plan_data["x_coords"][i] and plan_data["y_coords"][i]:
                # Create Point object with survey data
                point = Point(
                    name=plan_data["from"][i],
                    x_coord=float(plan_data["x_coords"][i]),
                    y_coord=float(plan_data["y_coords"][i]),
                    bearing_to_next=(
                        plan_data["bearing"][i]
                        if i < len(plan_data["bearing"])
                        else None
                    ),
                    distance_to_next=(
                        float(plan_data["distance"][i])
                        if plan_data["distance"][i]
                        else None
                    ),
                    next_point=plan_data["to"][i] if i < len(plan_data["to"]) else None,
                )

                # Convert coordinates to latitude/longitude
                lat, lon = self.ghana_grid_to_latlon(point.x_coord, point.y_coord)
                point.latitude = round(lat, 6)
                point.longitude = round(lon, 6)
                points.append(point)

        # Process boundary coordinates
        boundary_coords = []
        north_easterns = data["site_plan_data"]["north_easterns"]

        # Convert each boundary point
        for i in range(len(north_easterns["norths"])):
            north = float(north_easterns["norths"][i])
            east = float(north_easterns["easterns"][i])
            lat, lon = self.ghana_grid_to_latlon(north, east)
            boundary_coords.append(
                {
                    "point": f"Boundary_{i+1}",
                    "northing": north,
                    "easting": east,
                    "latitude": round(lat, 6),
                    "longitude": round(lon, 6),
                }
            )

        # Organize results into a structured format
        result = {
            "plot_info": {
                "plot_number": data["plot_number"],
                "area": float(data["area"]),
                "metric": data["metric"],
                "locality": data["locality"],
                "district": data["district"],
                "region": data["region"],
            },
            "survey_points": [
                {
                    "point_name": p.name,
                    "original_coords": {"x": p.x_coord, "y": p.y_coord},
                    "converted_coords": {
                        "latitude": p.latitude,
                        "longitude": p.longitude,
                    },
                    "next_point": {
                        "name": p.next_point,
                        "bearing": p.bearing_to_next,
                        "bearing_decimal": (
                            self.convert_dms_to_decimal(p.bearing_to_next)
                            if p.bearing_to_next
                            else None
                        ),
                        "distance": p.distance_to_next,
                    },
                }
                for p in points
            ],
            "boundary_points": boundary_coords,
        }

        return result


def process_and_display_results(json_data: str) -> dict:
    """
    Process JSON input data and return structured results.

    Args:
        json_data (str): JSON string or dictionary containing survey data

    Returns:
        dict: Processed and structured survey data
    """
    # Convert string input to dictionary if necessary
    data = json.loads(json_data) if isinstance(json_data, str) else json_data
    processor = LandDataProcessor()
    results = processor.process_land_data(data)
    return results

In [4]:
def main(json_data: str) -> dict:
    """
    Main entry point for processing land survey data.

    Args:
        json_data (str): JSON string containing survey data

    Returns:
        dict: Dictionary containing either:
            - 'status': 'success' and 'data': processed results
            - 'status': 'error' and 'message': error description

    Example:
        >>> data = '{...}'  # JSON string with survey data
        >>> result = main(data)
        >>> print(result['status'])
        'success'
    """
    try:
        results = process_and_display_results(json_data)
        return {"status": "success", "data": results}
    except Exception as e:
        return {"status": "error", "message": str(e)}

In [5]:
sample_data = {
    "owners": ["Transport Research & Education Centre, Kumasi (TRECK)"],
    "plot_number": "Plot No. 15B, Research Hills",
    "date": "02/02/2022",
    "area": "1.78",
    "metric": "Acres",
    "scale": "1:2500",
    "locality": "KNUST",
    "district": "OFORIKROM",
    "region": "ASHANTI",
    "other_location_details": "",
    "regional_number": "",
    "reference_number": "",
    "site_plan_data": {
        "plan_data": {
            "from": [
                "KNUST TREK 10/2021/1",
                "KNUST TREK 10/2021/2",
                "KNUST TREK 10/2021/3",
                "KNUST TREK 10/2021/4",
                "KNUST TREK 10/2021/5",
                "KNUST CEPR 10/2021/2",
                "KNUST CEPR 10/2021/3",
                "BOA CORS",
            ],
            "x_coords": [
                "724125.606",
                "724101.844",
                "724009.960",
                "724057.009",
                "724197.311",
                "724330.685",
                "724294.927",
                "732285.928",
            ],
            "y_coords": [
                "695158.748",
                "695149.339",
                "695129.526",
                "694842.085",
                "694820.230",
                "695135.853",
                "694811.094",
                "673148.096",
            ],
            "bearing": [
                "13°10'",
                "15°30'",
                "201°11'",
                "263°28'",
                "234°59'",
                "175°59'",
                "290°15'",
                "",
            ],
            "distance": [
                "175.9",
                "290.1",
                "24.2",
                "354.39",
                "351.0",
                "23.8",
                "299.7",
                "",
            ],
            "to": [
                "KNUST TREK 10/2021/2",
                "KNUST TREK 10/2021/3",
                "KNUST TREK 10/2021/4",
                "KNUST TREK 10/2021/5",
                "KNUST CEPR 10/2021/2",
                "KNUST CEPR 10/2021/3",
                "BOA CORS",
                "",
            ],
        },
        "north_easterns": {
            "norths": ["723600", "723600", "724500", "724500"],
            "easterns": ["694500", "695500", "694500", "695500"],
        },
    },
}

In [ ]:
results = main(sample_data)
# print(results['data']['survey_points'])
point_list = [coords["converted_coords"] for coords in results["data"]["survey_points"]]
# for coords in results['data']['survey_points']:
#         point_list.append(coords["converted_coords"])


results["data"]["point_list"] = point_list

import pprint as pp

pp.pprint(results["data"])

{'boundary_points': [{'easting': 694500.0,
                      'latitude': 10.92307,
                      'longitude': 3.107885,
                      'northing': 723600.0,
                      'point': 'Boundary_1'},
                     {'easting': 695500.0,
                      'latitude': 10.932089,
                      'longitude': 3.10801,
                      'northing': 723600.0,
                      'point': 'Boundary_2'},
                     {'easting': 694500.0,
                      'latitude': 10.922959,
                      'longitude': 3.116099,
                      'northing': 724500.0,
                      'point': 'Boundary_3'},
                     {'easting': 695500.0,
                      'latitude': 10.931978,
                      'longitude': 3.116224,
                      'northing': 724500.0,
                      'point': 'Boundary_4'}],
 'plot_info': {'area': 1.78,
               'district': 'OFORIKROM',
               'locality': 'KNUST',
    

In [ ]:
{
    "plot_info": {
        "plot_number": "15B",
        "area": 1.78,
        "metric": "Acres (0.72 Ha)",
        "locality": "KNUST",
        "district": "OFORIKROM",
        "region": "ASHANTI",
    },
    "survey_points": [
        {
            "point_name": "KNUST.TREK.10/2021/1",
            "original_coords": {"x": 724125.686, "y": 695158.748},
            "converted_coords": {
                "latitude": 6.6655264110112284,
                "longitude": -1.5645176483991616,
            },
            "next_point": {
                "name": null,
                "bearing": null,
                "bearing_decimal": null,
                "distance": [null],
            },
        },
        {
            "point_name": "KNUST.TREK.10/2021/2",
            "original_coords": {"x": 724103.844, "y": 695149.339},
            "converted_coords": {
                "latitude": 6.665466169857804,
                "longitude": -1.5645435212323173,
            },
            "next_point": {
                "name": null,
                "bearing": null,
                "bearing_decimal": null,
                "distance": [null],
            },
        },
        {
            "point_name": "KNUST.TREK.10/2021/3",
            "original_coords": {"x": 724089.96, "y": 695129.526},
            "converted_coords": {
                "latitude": 6.665427833607018,
                "longitude": -1.5645981041468533,
            },
            "next_point": {
                "name": null,
                "bearing": null,
                "bearing_decimal": null,
                "distance": [null],
            },
        },
        {
            "point_name": "KNUST.TREK.10/2021/4",
            "original_coords": {"x": 724057.009, "y": 694842.085},
            "converted_coords": {
                "latitude": 6.665336091508574,
                "longitude": -1.565390507351992,
            },
            "next_point": {
                "name": null,
                "bearing": null,
                "bearing_decimal": null,
                "distance": [null],
            },
        },
        {
            "point_name": "KNUST.TREK.10/2021/5",
            "original_coords": {"x": 724197.311, "y": 694820.23},
            "converted_coords": {
                "latitude": 6.665722790800521,
                "longitude": -1.565451206986715,
            },
            "next_point": {
                "name": null,
                "bearing": null,
                "bearing_decimal": null,
                "distance": [null],
            },
        },
        {
            "point_name": "KNUST.CEPB.10/2021/2",
            "original_coords": {"x": 724330.685, "y": 695135.853},
            "converted_coords": {
                "latitude": 6.666091456391503,
                "longitude": -1.5645814187533298,
            },
            "next_point": {
                "name": null,
                "bearing": null,
                "bearing_decimal": null,
                "distance": [null],
            },
        },
        {
            "point_name": "KNUST.CEPB.10/2021/3",
            "original_coords": {"x": 724294.927, "y": 694811.094},
            "converted_coords": {
                "latitude": 6.665991858417778,
                "longitude": -1.5654767041976154,
            },
            "next_point": {
                "name": null,
                "bearing": null,
                "bearing_decimal": null,
                "distance": [null],
            },
        },
        {
            "point_name": "SGA.CORS 2020 3",
            "original_coords": {"x": 732285.928, "y": 673148.096},
            "converted_coords": {
                "latitude": 6.687948270539181,
                "longitude": -1.625231712662081,
            },
            "next_point": {
                "name": null,
                "bearing": null,
                "bearing_decimal": null,
                "distance": [null],
            },
        },
    ],
    "boundary_points": [
        {
            "point": "Boundary_1",
            "northing": 694500,
            "easting": 723500,
            "latitude": 6.58393971,
            "longitude": -1.48629714,
        },
        {
            "point": "Boundary_2",
            "northing": 695500,
            "easting": 724500,
            "latitude": 6.58669912,
            "longitude": -1.48354311,
        },
    ],
    "point_list": [
        {
            "latitude": 5.645179059853662,
            "longitude": -0.08064102612052632,
            "ref_point": False,
        },
        {
            "latitude": 5.645631326601124,
            "longitude": -0.08044624012476795,
            "ref_point": False,
        },
        {
            "latitude": 5.656521316006493,
            "longitude": -0.00716395661312132,
            "ref_point": False,
        },
        {
            "latitude": 5.645169889723519,
            "longitude": -0.08047250230247535,
            "ref_point": False,
        },
        {
            "latitude": 5.645630322561624,
            "longitude": -0.08064810254736614,
            "ref_point": False,
        },
    ],
}

In [ ]:
{
    "plot_info": {
        "plot_number": null,
        "area": "1.55",
        "metric": "OR 0.63 HECTARE",
        "locality": "NTENSERE",
        "district": "ATWIMA NWABIAGYA",
        "region": "ASHANTI",
    },
    "survey_points": [
        {
            "point_name": "SMDA A001 191",
            "original_coords": {"x": 732467.436, "y": 671635.953},
            "converted_coords": {
                "latitude": 6.688443311147751,
                "longitude": -1.629401644294754,
            },
            "next_point": {
                "name": "SGA A8209/24/4",
                "bearing": "296°54'",
                "bearing_decimal": 296.9,
                "distance": [49300],
            },
        },
        {
            "point_name": "SGA A8209/24/1",
            "original_coords": {"x": 754804.309, "y": 627493.699},
            "converted_coords": {
                "latitude": 6.749845890901951,
                "longitude": -1.7512039571191538,
            },
            "next_point": {
                "name": "SGA A8209/24/2",
                "bearing": "014°19'",
                "bearing_decimal": 14.316666666666666,
                "distance": [365.3],
            },
        },
        {
            "point_name": "SGA A8209/24/2",
            "original_coords": {"x": 755158.342, "y": 627583.996},
            "converted_coords": {
                "latitude": 6.75082219139636,
                "longitude": -1.7509564680938294,
            },
            "next_point": {
                "name": "SGA A8209/24/3",
                "bearing": "100°25'",
                "bearing_decimal": 100.41666666666667,
                "distance": [191.6],
            },
        },
        {
            "point_name": "SGA A8209/24/3",
            "original_coords": {"x": 755123.687, "y": 627772.465},
            "converted_coords": {
                "latitude": 6.750727462302289,
                "longitude": -1.7504366197572163,
            },
            "next_point": {
                "name": "SGA A8209/24/4",
                "bearing": "196°22'",
                "bearing_decimal": 196.36666666666667,
                "distance": [371.1],
            },
        },
        {
            "point_name": "SGA A8209/24/4",
            "original_coords": {"x": 754767.641, "y": 627667.865},
            "converted_coords": {
                "latitude": 6.749745552285251,
                "longitude": -1.7507235415641202,
            },
            "next_point": {
                "name": "SGA A8209/24/1",
                "bearing": "281°55'",
                "bearing_decimal": 281.9166666666667,
                "distance": [178],
            },
        },
    ],
    "boundary_points": [
        {
            "point": "Boundary_1",
            "northing": 755500,
            "easting": 627000,
            "latitude": 6.75176151,
            "longitude": -1.75256828,
        },
        {
            "point": "Boundary_2",
            "northing": 755000,
            "easting": 627500,
            "latitude": 6.75038535,
            "longitude": -1.75118741,
        },
        {
            "point": "Boundary_3",
            "northing": 754500,
            "easting": 628000,
            "latitude": 6.74900919,
            "longitude": -1.74980655,
        },
    ],
    "point_list": [
        {"latitude": 6.688443311147751, "longitude": -1.629401644294754},
        {"latitude": 6.749845890901951, "longitude": -1.7512039571191538},
        {"latitude": 6.75082219139636, "longitude": -1.7509564680938294},
        {"latitude": 6.750727462302289, "longitude": -1.7504366197572163},
        {"latitude": 6.749745552285251, "longitude": -1.7507235415641202},
    ],
}

In [21]:
text_01 = "SMDA A001 191"
text_001 = "SGA.CORS 2020 3"

text_02 = "SGA A8209/24/4"
text_002 = "KNUST.CEPB.10/2021/3"

# Prompt

"Write a python function to be able to use regex to to identify text with pattern similar to 'SMDA A001 191', 'SMDA A001 19 1' or 'SGA.CORS 2020 3'"

<re.Match object; span=(0, 4), match='1.55'>
1.55


In [6]:
import re


def identify_pattern(text):
    """
    Identifies if a given text matches patterns like:
    - 'SMDA A001 191'
    - 'SMDA A001 19 1'
    - 'SGA.CORS 2020 3'

    :param text: The input string to check.
    :return: True if the text matches the pattern, False otherwise.
    """
    pattern = r"^[A-Z]+(?:\.[A-Z]+)?\s[A-Z0-9]+\s(?:\d+\s?)+$"
    return bool(re.match(pattern, text))


# Test examples
examples = [
    "SMDA A001 191",
    "SMDA A001 19 1",
    "SGA.CORS 2020 3",
    "INVALID 123",
    "SGX.CORSA 20203",
    "SGA A8209/24/4",
    "KNUST.CEPB.10/2021/3",
]

# Check the patterns
results = {example: identify_pattern(example) for example in examples}
print(results)

{'SMDA A001 191': True, 'SMDA A001 19 1': True, 'SGA.CORS 2020 3': True, 'INVALID 123': False, 'SGX.CORSA 20203': False, 'SGA A8209/24/4': False, 'KNUST.CEPB.10/2021/3': False}


In [ ]:
point_list = [
    {
        "latitude": 6.6655264110112284,
        "longitude": -1.5645176483991616,
        "ref_point": False,
    },
    {
        "latitude": 6.665466169857804,
        "longitude": -1.5645435212323173,
        "ref_point": False,
    },
    {
        "latitude": 6.665427833607018,
        "longitude": -1.5645981041468533,
        "ref_point": False,
    },
    {
        "latitude": 6.665336091508574,
        "longitude": -1.565390507351992,
        "ref_point": False,
    },
    {
        "latitude": 6.666091456391503,
        "longitude": -1.5645814187533298,
        "ref_point": False,
    },
    {
        "latitude": 6.665991858417778,
        "longitude": -1.5654767041976154,
        "ref_point": False,
    },
]

In [7]:
import os
import folium
from IPython.display import display, display_html, IFrame


def visualize_coordinates_in_notebook(point_list):
    """
    Visualize a list of coordinates on a map directly in a Jupyter Notebook.

    :param point_list: List of dictionaries containing latitude and longitude.
    """
    # Create a folium map centered around the average latitude and longitude
    center_lat = sum(point["latitude"] for point in point_list) / len(point_list)
    center_lon = sum(point["longitude"] for point in point_list) / len(point_list)
    map_ = folium.Map(location=[center_lat, center_lon], zoom_start=14)

    plot_polygon = folium.Polygon(
        locations=[(point["latitude"], point["longitude"]) for point in point_list],
        color="#2196F3",
        weight=2,
        fill=True,
        fill_color="#2196F3",
        fill_opacity=0.4,
    )

    plot_polygon.add_to(map_)

    # Add points to the map
    for point in point_list:
        folium.Marker(
            location=[point["latitude"], point["longitude"]],
            popup=f"Lat: {point['latitude']}, Lon: {point['longitude']}",
        ).add_to(map_)

    # Save the map to a temporary HTML file
    map_file = "map.html"
    map_.save(map_file)

    full_path = os.path.abspath(map_file)
    print(full_path)
    # display_html(IFrame(src=full_path, width=700, height=200))

In [ ]:
def rearrange_coordinates(point_list, sort_by="latitude"):
    """
    Rearrange a list of coordinates in order based on the given sorting criterion.

    :param point_list: List of dictionaries containing latitude and longitude.
    :param sort_by: The key to sort by ("latitude", "longitude", or "both").
    :return: Sorted list of coordinates.
    """
    if sort_by == "latitude":
        return sorted(point_list, key=lambda point: point["latitude"])
    elif sort_by == "longitude":
        return sorted(point_list, key=lambda point: point["longitude"])
    elif sort_by == "both":
        return sorted(
            point_list, key=lambda point: (point["latitude"], point["longitude"])
        )
    else:
        raise ValueError(
            "Invalid sort_by value. Use 'latitude', 'longitude', or 'both'."
        )

In [11]:
# Visualize sorted by latitude
sorted_by_latitude = sorted(point_list, key=lambda x: x["latitude"])
visualize_coordinates_in_notebook(sorted_by_latitude)

NameError: name 'point_list' is not defined

In [12]:
# Visualize sorted by latitude
sorted_by_longitude = rearrange_coordinates(point_list, sort_by="longitude")
visualize_coordinates_in_notebook(sorted_by_longitude)

NameError: name 'point_list' is not defined

In [13]:
# Sort by both latitude and longitude
sorted_by_both = rearrange_coordinates(point_list, sort_by="both")
visualize_coordinates_in_notebook(sorted_by_both)

NameError: name 'point_list' is not defined

In [26]:
import math


def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points on the Earth.
    :param lat1, lon1: Latitude and Longitude of point 1 in decimal degrees.
    :param lat2, lon2: Latitude and Longitude of point 2 in decimal degrees.
    :return: Distance in kilometers.
    """
    R = 6371  # Radius of Earth in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(math.radians(lat1))
        * math.cos(math.radians(lat2))
        * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c


def sort_by_distance(point_list):
    """
    Sort points by distance from the first point in the list.
    :param point_list: List of dictionaries containing latitude and longitude.
    :return: List of points sorted by distance from the first point.
    """
    if not point_list:
        return []

    # Get the reference point (the first point in the list)
    ref_point = point_list[0]
    ref_lat, ref_lon = ref_point["latitude"], ref_point["longitude"]

    # Calculate distance from the reference point and sort the list
    sorted_list = sorted(
        point_list,
        key=lambda point: haversine_distance(
            ref_lat, ref_lon, point["latitude"], point["longitude"]
        ),
    )
    return sorted_list


# Example usage


sorted_by_distance = sort_by_distance(point_list)
visualize_coordinates_in_notebook(point_list)

c:\Users\SethAntanah\Desktop\Projects\Other Projects\Streamlit Projects\landsearch\src\dev\map.html


In [ ]:
import math


def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points on the Earth.
    :param lat1, lon1: Latitude and Longitude of point 1 in decimal degrees.
    :param lat2, lon2: Latitude and Longitude of point 2 in decimal degrees.
    :return: Distance in kilometers.
    """
    R = 6371  # Radius of Earth in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(math.radians(lat1))
        * math.cos(math.radians(lat2))
        * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c


def order_points_by_proximity(point_list):
    """
    Order points by proximity chaining, starting with the first point.
    :param point_list: List of dictionaries with latitude and longitude.
    :return: List of points in the correct order based on proximity.
    """
    if not point_list:
        return []

    ordered_points = [point_list[0]]  # Start with the first point
    remaining_points = point_list[1:]  # Remaining points to visit

    while remaining_points:
        # Get the last point in the ordered list
        current_point = ordered_points[-1]
        current_lat, current_lon = current_point["latitude"], current_point["longitude"]

        # Find the nearest point among the remaining
        nearest_point = min(
            remaining_points,
            key=lambda point: haversine_distance(
                current_lat, current_lon, point["latitude"], point["longitude"]
            ),
        )

        # Add the nearest point to the ordered list and remove it from remaining points
        ordered_points.append(nearest_point)
        remaining_points.remove(nearest_point)

    return ordered_points


# Get ordered points
ordered_points = order_points_by_proximity(point_list)

# Print the results
visualize_coordinates_in_notebook(ordered_points)

NameError: name 'point_list' is not defined

In [15]:
def order_points_by_bearing(point_list):
    """
    Order points based on clockwise bearing from the first point.
    """
    ref_point = point_list[0]
    ref_lat, ref_lon = ref_point["latitude"], ref_point["longitude"]

    def bearing(point):
        lat, lon = point["latitude"], point["longitude"]
        angle = math.atan2(lon - ref_lon, lat - ref_lat)
        return (math.degrees(angle) + 360) % 360  # Normalize to [0, 360)

    return sorted(point_list, key=bearing)


# Get ordered points
ordered_points = order_points_by_bearing(point_list)

# Print the results
visualize_coordinates_in_notebook(ordered_points)

NameError: name 'point_list' is not defined

In [ ]:
import rtree
from shapely.geometry import Point


def reorder_points(points):
    """Reorders a list of points based on spatial proximity.

    Args:
        points: A list of dictionaries, each containing 'latitude' and 'longitude' keys.

    Returns:
        A list of reordered points.
    """
    # Create an R-tree index
    index = rtree.index.Index()
    for i, point in enumerate(points):
        index.insert(
            i,
            (
                point["longitude"],
                point["latitude"],
                point["longitude"],
                point["latitude"],
            ),
        )

    # Find the nearest neighbor for each point
    reordered_points = []
    visited = set()
    current_index = 0  # Start with the first point

    while len(reordered_points) < len(points):
        reordered_points.append(points[current_index])
        visited.add(current_index)

        # Find the nearest neighbor that has not been visited
        neighbors = index.nearest(
            (points[current_index]["longitude"], points[current_index]["latitude"]) * 2,
            len(points),
        )
        for neighbor in neighbors:
            if neighbor not in visited:
                current_index = neighbor
                break

    return reordered_points


# Example usage:
points = [
    {
        "latitude": 6.6655264110112284,
        "longitude": -1.5645176483991616,
        "ref_point": False,
    },
    {
        "latitude": 6.665466169857804,
        "longitude": -1.5645435212323173,
        "ref_point": False,
    },
    {
        "latitude": 6.665427833607018,
        "longitude": -1.5645981041468533,
        "ref_point": False,
    },
    {
        "latitude": 6.665336091508574,
        "longitude": -1.565390507351992,
        "ref_point": False,
    },
    {
        "latitude": 6.666091456391503,
        "longitude": -1.5645814187533298,
        "ref_point": False,
    },
    {
        "latitude": 6.665991858417778,
        "longitude": -1.5654767041976154,
        "ref_point": False,
    },
]

reordered_points = reorder_points(points)
# Print the results
visualize_coordinates_in_notebook(reordered_points)

c:\Users\SethAntanah\Desktop\Projects\Other Projects\Streamlit Projects\landsearch\src\dev\map.html


In [ ]:
data = [
    {
        "latitude": 5.777300184321659,
        "longitude": -0.09704742419819078,
        "ref_point": False,
    },
    {
        "latitude": 5.777552603198304,
        "longitude": -0.097168428017529,
        "ref_point": False,
    },
    {
        "latitude": 5.777632292682959,
        "longitude": -0.0969928984380644,
        "ref_point": False,
    },
    {
        "latitude": 5.77737982262644,
        "longitude": -0.09687439935416571,
        "ref_point": False,
    },
]

# Print the results
visualize_coordinates_in_notebook(data)

c:\Users\SethAntanah\Desktop\Projects\Other Projects\Streamlit Projects\landsearch\src\dev\map.html


In [ ]:
data = {
    "plot_info": {
        "plot_number": None,
        "area": 0.16,
        "metric": "Acres",
        "locality": "KATAMANSO",
        "district": "K.K.M.A",
        "region": "GREATER ACCRA",
        "owners": ["MRS NAA AYELEY BROWN"],
        "date": "22-09-2021",
        "scale": "1:2500",
        "other_location_details": "- Shewn Edged Pink -",
        "surveyors_name": "EDWIN ADDO-TAWIAH",
        "surveyors_location": None,
        "surveyors_reg_number": "388",
        "regional_number": None,
        "reference_number": None,
    },
    "survey_points": [
        {
            "point_name": "SGGA. F0258/21/1",
            "original_coords": {"x": 402081.98, "y": 1227948.15, "ref_point": False},
            "converted_coords": {
                "latitude": 5.777300184321659,
                "longitude": -0.09704742419819078,
                "ref_point": False,
            },
            "next_point": {
                "name": "SGGA. F0258/21/2",
                "bearing": "334°16'",
                "bearing_decimal": 334.26666666666665,
                "distance": [101.6],
            },
        },
        {
            "point_name": "SGGA. F0258/21/2",
            "original_coords": {"x": 402173.48, "y": 1227904.04, "ref_point": False},
            "converted_coords": {
                "latitude": 5.777552603198304,
                "longitude": -0.097168428017529,
                "ref_point": False,
            },
            "next_point": {
                "name": "SGGA. F0258/21/3",
                "bearing": "065°31'",
                "bearing_decimal": 65.51666666666667,
                "distance": [70],
            },
        },
        {
            "point_name": "SGGA. F0258/21/3",
            "original_coords": {"x": 402202.49, "y": 1227967.77, "ref_point": False},
            "converted_coords": {
                "latitude": 5.777632292682959,
                "longitude": -0.0969928984380644,
                "ref_point": False,
            },
            "next_point": {
                "name": "SGGA. F0258/21/4",
                "bearing": "154°44'",
                "bearing_decimal": 154.73333333333332,
                "distance": [101.2],
            },
        },
        {
            "point_name": "SGGA. F0258/21/4",
            "original_coords": {"x": 402110.97, "y": 1228010.97, "ref_point": False},
            "converted_coords": {
                "latitude": 5.77737982262644,
                "longitude": -0.09687439935416571,
                "ref_point": False,
            },
            "next_point": {
                "name": "SGGA. F0258/21/4",
                "bearing": "245°13′",
                "bearing_decimal": 245.21666666666667,
                "distance": [69.2],
            },
        },
        {
            "point_name": "SGGA. EX/TD/19/2",
            "original_coords": {"x": 358320.5, "y": 1260681.89, "ref_point": False},
            "converted_coords": {
                "latitude": 5.656521316006493,
                "longitude": -0.00716395661312132,
                "ref_point": False,
            },
            "next_point": {
                "name": "SGGA. F0258/21/1",
                "bearing": "323°12",
                "bearing_decimal": 323.2,
                "distance": [54],
            },
        },
    ],
    "boundary_points": [
        {
            "point": "Boundary_1",
            "northing": 402500,
            "easting": 1227500,
            "latitude": 5.77845444,
            "longitude": -0.09827904,
        },
        {
            "point": "Boundary_2",
            "northing": 402000,
            "easting": 1228000,
            "latitude": 5.77707397,
            "longitude": -0.09690508,
        },
        {
            "point": "Boundary_3",
            "northing": 401500,
            "easting": 1228500,
            "latitude": 5.7756935,
            "longitude": -0.09553112,
        },
    ],
    "point_list": [
        {
            "latitude": 5.777300184321659,
            "longitude": -0.09704742419819078,
            "ref_point": False,
        },
        {
            "latitude": 5.777632292682959,
            "longitude": -0.0969928984380644,
            "ref_point": False,
        },
        {
            "latitude": 5.77737982262644,
            "longitude": -0.09687439935416571,
            "ref_point": False,
        },
        {
            "latitude": 5.656521316006493,
            "longitude": -0.00716395661312132,
            "ref_point": False,
        },
        {
            "latitude": 5.777552603198304,
            "longitude": -0.097168428017529,
            "ref_point": False,
        },
    ],
}

points = data["point_list"]
reordered_points = reorder_points(points)
visualize_coordinates_in_notebook(reordered_points)

c:\Users\SethAntanah\Desktop\Projects\Other Projects\Streamlit Projects\landsearch\src\dev\map.html


In [ ]:
{
    "owners": ["MRS NAA AYELEY BROWN"],
    "plot_number": None,
    "date": "22-09-2021",
    "area": "0.16",
    "metric": "Acres",
    "scale": "1:2500",
    "locality": "KATAMANSO",
    "district": "K.K.M.A",
    "region": "GREATER ACCRA",
    "other_location_details": "Shewn Edged Pink",
    "surveyors_name": "EDWIN ADDO-TAWIAH",
    "surveyors_location": None,
    "surveyors_reg_number": "388",
    "regional_number": None,
    "reference_number": None,
    "site_plan_data": {
        "plan_data": {
            "from": [
                "SGGA. F0258/21/1",
                "SGGA. F0258/21/2",
                "SGGA. F0258/21/3",
                "SGGA. F0258/21/4",
                "SGGA. EX/TD/19/2",
            ],
            "x_coords": [
                "402081.98",
                "402173.48",
                "402202.49",
                "402110.97",
                "358320.50",
            ],
            "y_coords": [
                "1227948.15",
                "1227904.04",
                "1227967.77",
                "1228010.97",
                "1260681.89",
            ],
            "bearing": ["334°16'", "065°31'", "154°44'", "245°13′", "323°12"],
            "distance": ["101.6", "70.0", "101.2", "69.2", "54,649.5"],
            "to": [
                "SGGA. F0258/21/2",
                "SGGA. F0258/21/3",
                "SGGA. F0258/21/4",
                "SGGA. F0258/21/4",
                "SGGA. F0258/21/1",
            ],
            "ref": [True, True, True, True, False],
        },
        "north_easterns": {
            "norths": ["402500", "402000", "401500"],
            "easterns": ["1227500", "1228000", "1228500"],
        },
    },
}